<a href="https://colab.research.google.com/github/Uladzimir1367/Big-Data/blob/main/seminar2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Условие:
Загрузите датасет по ценам на жилье Airbnb, доступный на kaggle.com: https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data
Подсчитайте среднее значение и дисперсию по признаку ”price” в hive
Используя Python, реализуйте скрипт mapper.py и reducer.py для расчета
Проверьте правильность подсчета статистики методом mapreduce в сравнении со hive.

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('AB_NYC_2019.csv')
df

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,NaN,NaN,2,9
48891,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,NaN,NaN,2,36
48892,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,NaN,NaN,1,27
48893,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,NaN,NaN,6,2


# среднеe значение

In [ ]:
df['price'].mean()

152.7206871868289

# вычисление дисперсии

In [ ]:
df['price'].var()

57674.02524696099

In [ ]:
c = df['price']
c


0        149
1        225
2        150
3         89
4         80
        ... 
48890     70
48891     40
48892    115
48893     55
48894     90
Name: price, Length: 48895, dtype: int64

In [ ]:
c.to_csv('price.csv')
df1 = pd.read_csv('price.csv')
df1


,Unnamed: 0,price
0,0,149
1,1,225
2,2,150
3,3,89
4,4,80
...,...,...
48890,48890,70
48891,48891,40
48892,48892,115
48893,48893,55


In [ ]:

df1['price'].mean()

152.7206871868289

In [ ]:
df1['price'].var()

57674.02524696099

In [ ]:
df1['price'].max()

10000

Используя Python, реализуем скрипт mapper.py и reducer.py для расчета

In [17]:
import csv
import os
from pathlib import Path

# mapper.py

In [18]:
def mapper(path):

    if path.is_file():

        result = []

        with open(path, 'r', encoding='utf-8') as f:
            next(f)
            for line in f:
                data = line.strip().split(',')

                if len(data) > 9:
                    try:
                        price = int(data[9])
                        result.append(f"{price}\t1")
                    except ValueError:
                        continue
    else:
        print('Not found file')
    return result

# reducer.py

In [19]:
def reducer(input_data):

    sum_price = 0
    count = 0
    prices = []

    for line in input_data:
        price, cnt = line.strip().split('\t')

        try:
            price = int(price)
            cnt = int(cnt)
        except ValueError:
            continue

        sum_price += price * cnt
        count += cnt
        prices.append(price)


    if count == 0:
        print("Нет данных для расчета.")
        return None
    else:
        mean_price = sum_price / count
        variance = sum((x - mean_price) ** 2 for x in prices) / count

        return mean_price, variance

In [20]:
mapped_data = mapper(Path('AB_NYC_2019.csv'))
mean_price, variance = reducer(mapped_data)
print(mean_price, variance)

153.23297422544968 59124.1250938161
